# Get one symbol nakeds for `NSE`
***

# !!! `REMEMBER TO REMOVE PREVIOUS EARLIEST NAKEDS PICKLES` !!!

THIS NEEDS TO CHANGE TO `EARLIEST NAKEDS` !!!

- [x] get equity fno list
- [x] get lot-size
- [x] get market price of options
- [x] get volatilities for chains
- [x] get all chains with dte for earliest nakeds

***
 - [ ] pack all into symbol objects



In [1]:
## THIS CELL SHOULD BE IN ALL VSCODE NOTEBOOKS ##

MARKET = "NSE"

# Set the root
from from_root import from_root

ROOT = from_root()

import pandas as pd
from loguru import logger

pd.options.display.max_columns = None
pd.set_option("display.precision", 2)

import sys
from pathlib import Path

# Add `src` and ROOT to _src.pth in .venv to allow imports in VS Code
from sysconfig import get_path

if "src" not in Path.cwd().parts:
    src_path = str(Path(get_path("purelib")) / "_src.pth")
    with open(src_path, "w") as f:
        f.write(str(ROOT / "src\n"))
        f.write(str(ROOT))
        if str(ROOT) not in sys.path:
            sys.path.insert(1, str(ROOT))

# Start the Jupyter loop
from ib_async import util

util.startLoop()

# Imports

In [2]:
# --- IMPORTS ---
# ---------------

import asyncio
import glob
import io
import json
import logging  # for ib_async log redirection
import math
import os
import pickle
import re
from dataclasses import dataclass
from datetime import date, datetime, time, timedelta, timezone
from itertools import chain
from pathlib import Path
from typing import List, Union

import numpy as np
import pandas as pd
import pytz
import requests
import yaml
from bs4 import BeautifulSoup
from dotenv import load_dotenv
from from_root import from_root
from ib_async import IB, LimitOrder, MarketOrder, Option, Order, util
from loguru import logger
from pandas import json_normalize
from scipy.integrate import quad
from scipy.stats import norm
from tqdm import tqdm

# Functions

## Tools (Utilties)

In [3]:
# --- TOOLS ---
# -------------


def load_config():
    """Loads configuration from .env and config.yml files."""

    # Load environment variables from .env file
    load_dotenv()

    # Load config from YAML file

    ROOT = from_root()
    with open(ROOT / "config" / "config.yml", "r") as f:
        config = yaml.safe_load(f)

    # Merge environment variables with config
    for key, value in os.environ.items():
        if key in config:
            config[key] = value

    return config


def delete_files(file_paths):
    """Deletes a list or set of files.

    Args:
      file_paths: A list or set of file paths as strings.
    """

    for file_path in file_paths:
        try:
            if os.path.exists(file_path):
                os.remove(file_path)
                print(f"Deleted: {file_path}")
            else:
                print(f"File not found: {file_path}")
        except OSError as e:
            print(f"Error deleting file: {file_path}, {e}")


def get_pickle_suffix(pattern: str = "/*nakeds*"):
    """Checks pickle file name suffix and gets the next digit"""

    def extract_suffix(file):
        try:
            return int(file[:-4][-1])
        except ValueError:
            return 0

    files = get_files_from_patterns(pattern)
    suffixes = [extract_suffix(file) for file in files]
    return max(suffixes) + 1 if suffixes else 1


def yes_or_no(question, default="n") -> bool:
  """Asks a yes or no question with a default answer.

  Args:
    question: The question to ask.
    default: The default answer if the user presses Enter.

  Returns:
    True if the user answered yes, False otherwise.
  """

  while True:
    answer = input(question + f" (y/n): ").lower().strip()
    if not answer:
      return default == "y"
    if answer in ("y", "yes"):
      return True
    elif answer in ("n", "no"):
      return False
    else:
      print("Please answer yes or no.")


def remove_raw_nakeds(save: bool=True):

    # consolidate and pickle
    files = get_files_from_patterns(pattern="*nakeds*")

    if files:
        df_nakeds = pd.concat([get_pickle(f) for f in files], ignore_index=True)

        # save a df_nakeds for rough use
        if save:
            pickle_me(df_nakeds, ROOT / "data" / "df_nakeds.pkl")

            # historize
            filename = f"{datetime.now().strftime('%Y%m%d_%I_%M_%p')}_naked_orders.pkl"
            pickle_me(df_nakeds, str(ROOT / "data" / "xn_history" / str(filename)))

        # delete consolidated files
        delete_files(files)
        logger.info(f"Deleted files {files}")


def get_files_from_patterns(log_path = None, pattern: str = "*nakeds*") -> list:
    """Gets list of files of matching pattern from a folder path """

    ROOT = from_root()

    if log_path is None: # Defaults to raw data folder
        log_path = ROOT / 'data' / 'raw'

    result = glob.glob(str(log_path / pattern))

    return result


def split_dates(days: int = 365, chunks: int = 50) -> list:
    """splits dates into buckets, based on chunks"""

    end = datetime.today()
    periods = int(days / chunks)
    start = end - timedelta(days=days)

    if days < chunks:
        date_ranges = [(start, end)]
    else:
        dates = pd.date_range(start, end, periods).date
        date_ranges = list(
            zip(pd.Series(dates), pd.Series(dates).shift(-1) + timedelta(days=-1))
        )[:-1]

    # remove last tuple having period as NaT
    if any(pd.isna(e) for element in date_ranges for e in element):
        date_ranges = date_ranges[:-1]

    return date_ranges


def clean_ib_util_df(contracts: Union[list, pd.Series]) -> pd.DataFrame:
    """Cleans ib_async's util.df to keep only relevant columns"""

    df1 = pd.DataFrame([])  # initialize

    if isinstance(contracts, list):
        df1 = util.df(contracts)
    elif isinstance(contracts, pd.Series):
        try:
            contract_list = list(contracts)
            df1 = util.df(contract_list)  # it could be a series
        except (AttributeError, ValueError):
            logger.error(f"cannot clean type: {type(contracts)}")
    else:
        logger.error(f"cannot clean unknowntype: {type(contracts)}")

    if not df1.empty:

        df1.rename(
            {"lastTradeDateOrContractMonth": "expiry"}, axis="columns", inplace=True
        )

        df1 = df1.assign(expiry=pd.to_datetime(df1.expiry))
        cols = list(df1.columns[:6])
        cols.append("multiplier")
        df2 = df1[cols]
        df2 = df2.assign(contract=contracts)

    else:
        df2 = None

    return df2


def convert_to_utc_datetime(date_string, eod=False):
    """Converts nse date strings to utc datetimes. If eod is chosen 3:30 PM IST is taken."""

    # List of possible date formats
    date_formats = ["%d-%b-%Y", "%d %b %Y", "%Y-%m-%d %H:%M:%S.%f%z"]

    for date_format in date_formats:
        try:
            dt = datetime.strptime(date_string, date_format)

            # If the parsed datetime doesn't have timezone info, assume it's UTC
            if dt.tzinfo is None:
                dt = dt.replace(tzinfo=pytz.UTC)
            else:
                # If it has timezone info, convert to UTC
                dt = dt.astimezone(pytz.UTC)

            if eod:
                # Set time to 3:30 PM India time for all formats when eod is True
                india_time = time(hour=15, minute=30)
                india_tz = pytz.timezone("Asia/Kolkata")
                dt = india_tz.localize(datetime.combine(dt.date(), india_time))
                dt = dt.astimezone(pytz.UTC)
            elif dt.time() == time(0, 0):  # If time is midnight (00:00:00)
                # Keep it as midnight UTC
                dt = dt.replace(hour=0, minute=0, second=0, microsecond=0)

            return dt
        except ValueError:
            continue

    # If none of the formats work, raise an error
    raise ValueError(f"Unable to parse date string: {date_string}")


def convert_to_numeric(col: pd.Series):
    """convert to numeric if possible, only for object dtypes"""

    if col.dtype == "object":
        try:
            return pd.to_numeric(col)
        except ValueError:
            return col
    return col


def convert_daily_volatility_to_yearly(daily_volatility, days: float = 252):
    return daily_volatility * math.sqrt(days)


def find_closest_strike(df, above=False):
    """
    Finds the row with the strike closest to the undPrice.

    Parameters:
    df (pd.DataFrame): The input DataFrame.
    above (bool): If True, find the closest strike above undPrice. If False, find the closest strike below undPrice.

    Returns:
    pd.DataFrame: A DataFrame with the single row that has the closest strike.
    """
    undPrice = df["undPrice"].iloc[0]  # Get the undPrice from the first row

    if above:
        # Filter for strikes above undPrice
        mask = df["strike"] > undPrice
    else:
        # Filter for strikes below undPrice
        mask = df["strike"] < undPrice

    if not mask.any():
        return pd.DataFrame()  # Return an empty DataFrame if no rows match the criteria

    # Calculate the absolute difference between strike and undPrice
    diff = np.abs(df.loc[mask, "strike"] - undPrice)

    # Find the index of the row with the minimum difference
    closest_index = diff.idxmin()

    # Return the closest row as a DataFrame
    return df.loc[[closest_index]]


def get_dte(s: pd.Series) -> pd.Series:
    """Gets days to expiry. Expects series of UTC timestamps"""

    now_utc = datetime.now(pytz.UTC)
    return (s - now_utc).dt.total_seconds() / (24 * 60 * 60)


def fbfillnas(ser: pd.Series) -> pd.Series:
    """Fills nan in series forwards first and then backwards"""

    s = ser.copy()

    # Find the first non-NaN value
    first_non_nan = s.dropna().iloc[0]

    # Fill first NaN with the first non-NaN value
    s.iloc[0] = first_non_nan

    # Fill remaining NaN values with the next valid value
    s = s.fillna(s.bfill())

    # Fill remaining NaN values with the previous valid value
    s = s.fillna(s.ffill())

    return ser.fillna(s)


def get_a_stdev(iv: float, price: float, dte: float) -> float:
    """Gives 1 Standard Deviation value for annual iv"""

    return iv * price * math.sqrt(dte / 365)


def get_prob(sd):
    """Compute probability of a normal standard deviation

    Arg:
        (sd) as standard deviation
    Returns:
        probability as a float

    """
    prob = quad(lambda x: np.exp(-(x**2) / 2) / np.sqrt(2 * np.pi), -sd, sd)[0]
    return prob


def get_prec(v: float, base: float) -> float:
    """Gives the precision value

    Args:
       (v) as value needing precision in float
       (base) as the base value e.g. 0.05
    Returns:
        the precise value"""

    try:
        output = round(round((v) / base) * base, -int(math.floor(math.log10(base))))
    except Exception:
        output = None

    return output


def make_contracts_orders(
    df: pd.DataFrame, EXCHANGE: str = "NSE", action: str = "SELL"
) -> pd.DataFrame:
    """Makes df[contract, order] from option df. Used for margin check"""

    df_out = pd.DataFrame(
        {
            "contract": df.apply(
                lambda row: Option(
                    row.ib_symbol,
                    # util.formatIBDatetime(row.expiry.date())[:8],
                    util.formatIBDatetime(row.expiry.date()),
                    row.strike,
                    row.right,
                    EXCHANGE,
                ),
                axis=1,
            ),
            "order": df.apply(
                lambda row: MarketOrder(action=action, totalQuantity=row.lot), axis=1
            ),
        }
    )

    return df_out


def black_scholes(
    S: float,  # underlying
    K: float,  # strike
    T: float,  # years-to-expiry
    r: float,  # risk-free rate
    sigma: float,  # implied volatility
    option_type: str,  # Put or Call right
) -> float:
    """Black-Scholes Option Pricing Model"""

    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    if option_type == "C":
        price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    elif option_type == "P":
        price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    else:
        raise ValueError("Invalid option type. Use 'C' for Call and 'P' for Put.")

    return price


def arrange_df_columns(df: pd.DataFrame, col_map: str) -> list:
    """
    Arranges columns based on col_map given
    """

    cols = []

    if isinstance(col_map, str):
        v = re.split(r"[,\t\s]+", col_map)
        cols = [s for s in v if s in df.columns]

    return cols


def arrange_orders(df: pd.DataFrame, **kwargs) -> pd.DataFrame:
    """Arranges orders to get the best ROM and filters out excessive margins
    
    Usage:
      arrange_orders(df, 
                   maxmargin=MAXMARGINPERORDER, 
                   how_many=2,
                   puts_only=True)"""

    MARKET = df.contract.iloc[0].exchange

    # Takes maxmargin if it is provided. Else defaults it to 5,000 for SNP
    maxmargin = kwargs.get('maxmargin', 5000)
    how_many = kwargs.get('how_many', 2)
    puts_only = kwargs.get('puts_only', False)

    # files = get_files_from_patterns("/*nakeds*")


    # Sort by safe_strike: strike ratio to get the safest calls and puts

    # ... group calls

    if not puts_only:
        gc = (
            df[df.right == "C"]
            .assign(ratio=df.safe_strike / df.strike)
            .sort_values("ratio")
            .groupby("ib_symbol")
        )
        df_calls = gc.head(how_many).sort_values(
            ["ib_symbol", "strike"], ascending=[True, False]
        )
        dfc = df_calls[df_calls.margin < maxmargin]
        dfc = dfc.assign(ratio=dfc.safe_strike / dfc.strike).sort_values("ratio")
    else:
        dfc = pd.DataFrame([])

    # ... group puts
    gc = (
        df[df.right == "P"]
        .assign(ratio=df.strike / df.safe_strike)
        .sort_values("ratio")
        .groupby("ib_symbol")
    )
    df_puts = gc.head(how_many).sort_values(
        ["ib_symbol", "strike"], ascending=[True, False]
    )
    dfp = df_puts[df_puts.margin < maxmargin]
    dfp = dfp.assign(ratio=dfp.strike / dfp.safe_strike).sort_values("ratio")

    # ... prepare the nakeds to order
    df_nakeds = pd.concat([dfc, dfp], axis=0, ignore_index=True)

    # ... remove dubious xPrice which are less than option price
    df_nakeds = df_nakeds[df_nakeds.xPrice > df_nakeds.price]

    df_nakeds = df_nakeds.reset_index(drop=True)

    # Sort the DataFrame by 'xPrice/price' in ascending order
    df_nakeds = df_nakeds.loc[
        df_nakeds["xPrice"].
            div(df_nakeds["price"]).
            sort_values().index
    ]

    return df_nakeds


# --- PICKLE TOOLS ----
# -------------------------


def pickle_me(obj, file_name_with_path: Path):
    """Pickles objects in a given path"""

    with open(str(file_name_with_path), "wb") as handle:
        pickle.dump(obj, handle, protocol=pickle.HIGHEST_PROTOCOL)


def get_pickle(path: Path, print_msg: bool = True):
    """Gets pickled object"""

    output = None  # initialize

    try:
        with open(path, "rb") as f:
            output = pickle.load(f)
    except FileNotFoundError:
        if print_msg:
            logger.error(f"file not found: {path}")

    return output

## IB functions

In [15]:
# ---- IB FUNCTIONS ----
# ----------------------


def get_ib_margin(contract: Option, order: MarketOrder, port: int) -> dict:
    """Gets margin and commission of a contract"""

    with IB().connect(port=port) as ib:
        if contract.conId == 0:  # qualify raw contracts
            contract = next(iter(ib.qualifyContracts(contract)))
        wif = ib.whatIfOrder(contract, order)

    # margin = float(wif.initMarginChange) # initial margin is too high compared to Zerodha, SAMCO
    margin = float(wif.maintMarginChange)
    comm = min(float(wif.commission), float(wif.minCommission), float(wif.maxCommission))
    if comm > 1e7:
        comm = np.nan

    return {"contract": contract, "margin": margin, "comm": comm}


def get_ib_margin_comms(df: pd.DataFrame, port: int) -> pd.DataFrame:
    """Qualified Contracts, Margins and Commissions from an options df"""

    symbol = df.ib_symbol.iloc[0]
    df_cos = make_contracts_orders(df)

    cts = [d if d.conId == 0 else None for d in df_cos.contract]
    with IB().connect(port=port) as ib:
        if len(cts) > 40:
            ib.qualifyContracts(*tqdm(cts, desc=f"Qualifying {symbol} options"))
        else:
            ib.qualifyContracts(*cts)

        df_cos.contract = cts
        ib.disconnect()

    if len(df_cos) > 1:  # use tqdm.pandas.progress_apply()
        tqdm.pandas(desc=f"Calculating {symbol} margins")
        data = df_cos.progress_apply(
            lambda row: get_ib_margin(row.contract, row.order, port=port), axis=1
        )
    else:
        data = df_cos.apply(lambda row: get_ib_margin(row.contract, row.order, port=port), axis=1)

    df_mcom = pd.DataFrame.from_dict(data.to_dict()).T

    # replace raw contracts with qualified
    df_q = df_cos.join(df_mcom, how="outer", lsuffix="_left").drop(
        ["contract_left", "order"], axis=1
    )

    # merge margins and commissions
    df_opts = df.merge(df_q, left_index=True, right_index=True, suffixes=('_left', ''))
    df_opts = df_opts.drop(columns='contract_left', errors='ignore')

    # determine the secType for IB
    df_opts = df_opts.assign(secType=df_opts.contract.apply(lambda s: s.secType))

    return df_opts


# --- ORDER HANDLING ---
# -----------------------


def place_orders(ib: IB, cos: Union[tuple, list], blk_size: int = 25) -> List:
    """!!!CAUTION!!!: This places orders in the system
    ---
    NOTE: cos could be a single (contract, order)
          or a tuple/list of ((c1, o1), (c2, o2)...)
          made using tuple(zip(cts, ords))
    ---
    USAGE:
    ---
    cos = tuple((c, o) for c, o in zip(contracts, orders))
    with IB().connect(port=port) as ib:
        ordered = place_orders(ib=ib, cos=cos)
    """

    trades = []

    if isinstance(cos, (tuple, list)) and (len(cos) == 2):
        c, o = cos
        trades.append(ib.placeOrder(c, o))

    else:
        cobs = {cos[i : i + blk_size] for i in range(0, len(cos), blk_size)}

        for b in tqdm(cobs):
            for c, o in b:
                td = ib.placeOrder(c, o)
                trades.append(td)
            ib.sleep(0.75)

    return trades


def get_open_orders(ib, is_active: bool = False) -> pd.DataFrame:
    """Gets open orders - blocking version"""

    ACTIVE_STATUS = "ApiPending, PendingSubmit, PreSubmitted, Submitted".split(",")

    df_openords = OpenOrder().empty()  # Initialize open orders

    trades = ib.reqAllOpenOrders()
    # trades = ib.trades()
    # ib.sleep(1) # time to take in all open orders

    if trades:

        all_trades_df = (
            clean_ib_util_df([t.contract for t in trades])
            .join(util.df(t.orderStatus for t in trades))
            .join(util.df(t.order for t in trades), lsuffix="_")
        )

        order = pd.Series([t.order for t in trades], name="order")

        all_trades_df = all_trades_df.assign(order=order)

        all_trades_df.rename(
            {"lastTradeDateOrContractMonth": "expiry"}, axis="columns", inplace=True
        )

        # all_trades_df = all_trades_df[all_trades_df.status.isin(ACTIVE_STATUS)]

        trades_cols = df_openords.columns

        dfo = all_trades_df[trades_cols]

        if is_active:
            dfo = dfo[dfo.status.isin(ACTIVE_STATUS)]

        # dfo = dfo.assign(expiry=pd.to_datetime(dfo.expiry))

    return dfo


def quick_pf(ib) -> Union[None, pd.DataFrame]:
    """Gets the portfolio dataframe"""
    pf = ib.portfolio()  # returns an empty [] if there is nothing in the portfolio

    if pf != []:
        df_pf = util.df(pf)
        df_pf = (util.df(list(df_pf.contract)).iloc[:, :6]).join(
            df_pf.drop(columns=["account"])
        )
        df_pf = df_pf.rename(
            columns={
                "lastTradeDateOrContractMonth": "expiry",
                "marketPrice": "mktPrice",
                "marketValue": "mktVal",
                "averageCost": "avgCost",
                "unrealizedPNL": "unPnL",
                "realizedPNL": "rePnL",
            }
        )
    else:
        df_pf = Portfolio().empty()

    return df_pf


def cancel_all(port: int):
    """Cancels all orders"""

    with IB().connect(port=port, clientId=10) as ib:
        ib.reqGlobalCancel()




### Set logger for ib_async logs

In [5]:
# --- REDIRECT IB_ASYNC LOGGER ---


class LoguruHandler(logging.Handler):
    def emit(self, record):
        # Convert logging level to Loguru level
        loguru_level = logger.level(record.levelname)
        logger.log(loguru_level, record.getMessage(), record.args)


# Replace ib_async log handler
root_logger = logging.getLogger()
root_logger.handlers = [LoguruHandler()]

# Set the loguru logger
logger.add(sink=ROOT / "log" / "nakeds.log", mode="w")

1

## NSE specific functions

In [6]:
# --- NSE FUNCTIONS ---
# ---------------------

def make_earliest_nakeds(fnos: Union[List, set], 
                         save: bool = False) -> pd.DataFrame:
    """Make all early fnos"""

    timer = Timer("Making earliest nakeds")
    timer.start()

    dfs = []
    suffix = get_pickle_suffix(pattern="/*nakeds*")

    for symbol in fnos:

        try:
            df_nakeds = make_early_opts_for_symbol(symbol, port=port)
            df_nakeds = df_nakeds[df_nakeds.xPrice > 0]

        except (AttributeError, ValueError) as e:
            logger.error(e)
            df_nakeds = None

        dfs.append(df_nakeds)

        # collect dfs and save
        if dfs:
            try:
                df = pd.concat(dfs, axis=0, ignore_index=True)
            except ValueError as e:
                logger.error(f"No dfs to concat!. Error: {e}")
                df = pd.DataFrame([])

            if save and not df.empty:
                filename = str(f"earliest_nakeds{suffix}.pkl")
                pickle_me(df, ROOT / "data" / "raw" / filename)
        else:
            df = dfs

    timer.stop()

    return df


def make_early_opts_for_symbol(
    symbol: str,  # nse_symbol. can be equity or index
    port: int,
        ) -> pd.DataFrame:
    """Make target options for nakeds with earliest expiry
    
    Args:
       symbol: can be equity or index
       port: int. Could be LIVE_PORT | PAPER_PORT
       """

    # Instantiate nse
    nse = NSEfnos()

    # Get the basic quote
    fno_quote = nse.stock_quote_fno(symbol)

    # make the fno df with iv, hv, lot
    df_fno = equity_iv_df(fno_quote)

    # get margins and commissions from ib
    df_mcom = get_ib_margin_comms(df_fno, port=port)

    # Remove zero IVs
    df_mcom = df_mcom[df_mcom.iv > 0]

    # Remove zero DTEs and create a new df
    df = df_mcom[df_mcom.dte > 0]

    # Get the risk free rate
    rbi = RBI()
    risk_free_rate = rbi.repo_rate() / 100

    # Compute the black_scholes of option strike
    bsPrice = df_mcom.apply(
        lambda row: black_scholes(
            S=row["undPrice"],
            K=row["strike"],
            T=row["dte"] / 365,  # Convert days to years
            r=risk_free_rate,
            sigma=row["iv"],
            option_type=row["right"],
        ),
        axis=1,
    )

    # Compute the black_scholes price of safe_strike
    # get the safe-strike, based on std multiples
    df_sp = pd.concat(
        [
            df,
            pd.Series(
                df.iv
                * df.undPrice
                * (df.dte / 365).apply(lambda x: math.sqrt(x) if x >= 0 else np.nan),
                name="sdev",
            ),
        ],
        axis=1,
    )

    # calculate safe strike with option price added
    safe_strike = np.where(
        df_sp.right == "P",
        (df_sp.undPrice - df_sp.sdev * PUTSTDMULT).astype("int"),
        (df_sp.undPrice + df_sp.sdev * CALLSTDMULT).astype("int"),
    )

    df_sp = df_sp.assign(safe_strike=safe_strike)

    # intrinsic value
    intrinsic = np.where(
        df_sp.right == "P",
        (df_sp.strike - df_sp.safe_strike).map(lambda x: max(0, x)),
        (df_sp.safe_strike - df_sp.strike).map(lambda x: max(0, x)),
    )

    df_sp = df_sp.assign(intrinsic=intrinsic)

    # compute xPrice based on distance from safe_strike
    maxPrice = np.maximum(df_sp.price, pd.Series(bsPrice))

    # derive expected price from safe_strike
    df = df_sp.assign(
        xPrice=(df_sp.intrinsic + maxPrice).apply(
            lambda x: max(get_prec(x, 0.05), 0.05)
        )
    )

    # prevent divide-by-zero error for rom
    df = df.assign(margin=np.where(df.margin <= 0, np.nan, df.margin))

    # calculate the return on margin (rom)
    df = df.assign(rom=df.xPrice * df.lot / df.margin * 365 / df.dte)

    # Sort by likeliest
    df_out = df.loc[(df.xPrice / df.price).sort_values().index]

    return df_out.reset_index(drop=True)



def nse_ban_list() -> list:
    """Gets scrips banned today"""

    url = "https://nsearchives.nseindia.com/content/fo/fo_secban.csv"
    headers = {
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 "
        "(KHTML, like Gecko) "
        "Chrome/80.0.3987.149 Safari/537.36",
        "accept-language": "en,gu;q=0.9,hi;q=0.8",
        "accept-encoding": "gzip, deflate, br",
    }

    base_url = "https://www.nseindia.com"

    with requests.Session() as session:
        request = session.get(base_url, headers=headers, timeout=5)
        cookies = dict(request.cookies)
        response = session.get(url, headers=headers, timeout=5, cookies=cookies)

    df = pd.read_csv(io.StringIO(response.text))
    ban_list = df.iloc[:, 0].tolist()

    return ban_list


def live_cache(app_name):
    """Caches the output for time_out specified. This is done in order to
    prevent hitting live quote requests to NSE too frequently. This wrapper
    will fetch the quote/live result first time and return the same result for
    any calls within 'time_out' seconds.

    Logic:
        key = concat of args
        try:
            cached_value = self._cache[key]
            if now - self._cache['tstamp'] < time_out
                return cached_value['value']
        except AttributeError: # _cache attribute has not been created yet
            self._cache = {}
        finally:
            val = fetch-new-value
            new_value = {'tstamp': now, 'value': val}
            self._cache[key] = new_value
            return val

    """

    def wrapper(self, *args, **kwargs):
        """Wrapper function which calls the function only after the timeout,
        otherwise returns value from the cache.

        """
        # Get key by just concating the list of args and kwargs values and hope
        # that it does not break the code :P
        inputs = [str(a) for a in args] + [str(kwargs[k]) for k in kwargs]
        key = app_name.__name__ + "-".join(inputs)
        now = datetime.now()
        time_out = self.time_out
        try:
            cache_obj = self._cache[key]
            if now - cache_obj["timestamp"] < timedelta(seconds=time_out):
                return cache_obj["value"]
        except:
            self._cache = {}
        value = app_name(self, *args, **kwargs)
        self._cache[key] = {"value": value, "timestamp": now}
        return value

    return wrapper


def clean_stock_history(result: list) -> pd.DataFrame:
    """Cleans output of"""

    df = pd.concat(
        [pd.DataFrame(r.get("data")) for r in result], axis=0, ignore_index=True
    )

    # ...clean columns

    mapping = {
        "CH_SYMBOL": "nse_symbol",
        "TIMESTAMP": "date",
        "CH_OPENING_PRICE": "open",
        "CH_TRADE_HIGH_PRICE": "high",
        "CH_TRADE_LOW_PRICE": "low",
        "CH_CLOSING_PRICE": "close",
        "CH_TOT_TRADED_QTY": "qty_traded",
        "CH_TOT_TRADED_VAL": "value_traded",
        "CH_TOTAL_TRADES": "trades",
        "VWAP": "vwap",
        "updatedAt": "extracted_on",
    }

    df = df[[col for col in mapping.keys() if col in df.columns]].rename(
        columns=mapping
    )

    # ...convert column datatypes

    astype_map = {
        **{
            k: "float"
            for k in ["open", "high", "low", "close", "value_traded", "trades", "vwap"]
        },
        **{"qty_traded": "int"},
    }

    df = df.astype(astype_map)

    # ...change date columns to utc

    replace_cols = ["date", "extracted_on"]
    df1 = df[replace_cols].map(lambda x: datetime.fromisoformat(x))
    df = df.assign(date=df1.date, extracted_on=df1.extracted_on)

    return df


def clean_index_history(results: list) -> pd.DataFrame:
    """cleans index history and builds it as a dataframe"""

    df = pd.concat(
        [pd.DataFrame(json.loads(r.get("d"))) for r in results], ignore_index=True
    )

    # clean the df

    # ...drop unnecessary columns

    df = df.drop(df.columns[[0, 1]], axis=1)

    # ...rename
    df.columns = ["nse_symbol", "date", "open", "high", "low", "close"]

    # ...convert nse_symbol to IB's symbol
    df = pd.concat(
        [
            df.nse_symbol.map(IDXHISTSYMMAP).rename("symbol"),
            df,
        ],
        axis=1,
    )

    utc_dates = df.date.apply(lambda x: convert_to_utc_datetime(x, eod=True))

    df = df.assign(date=utc_dates)

    # .....convert ohlc to numeric
    convert_dict = {k: "float" for k in ["open", "high", "low", "close"]}

    df = df.astype(convert_dict)

    # .....sort by date
    df.sort_values(["nse_symbol", "date"], inplace=True, ignore_index=True)

    # .....add extract_date
    now = datetime.now()
    utc_now = now.astimezone(timezone.utc)
    df = df.assign(extracted_on=utc_now)

    return df


def nse2ib(nse_list):
    """Converts nse to ib friendly symbols"""

    subs = {"M&M": "MM", "M&MFIN": "MMFIN", "L&TFH": "LTFH", "NIFTY": "NIFTY50"}

    list_without_percent_sign = list(map(subs.get, nse_list, nse_list))

    # fix length to 9 characters
    ib_equity_fnos = [s[:9] for s in list_without_percent_sign]

    return ib_equity_fnos


def make_date_range_for_stock_history(
    symbol: str, days: int = 365, chunks: int = 50
) -> list:
    """Uses `split_dates` to make date range for stock history"""

    date_ranges = split_dates(days=days, chunks=chunks)

    series = "EQ"

    ranges = [
        {
            "symbol": symbol,
            "from": start.strftime("%d-%m-%Y"),
            "to": end.strftime("%d-%m-%Y"),
            "series": f'["{series}"]',
        }
        for start, end in date_ranges
    ]

    return ranges


def equity_iv_df(quotes: dict) -> pd.DataFrame:
    """Build a core df with symbol, undPrice, expiry, strike, volatilities, lot and price."""

    flat_data = json_normalize(quotes, sep="-")

    # get symbol, lot and underlying pricefrom quote

    symbol = quotes.get("info").get("symbol")

    lot = (
        quotes["stocks"][0].get("marketDeptOrderBook").get("tradeInfo").get("marketLot")
    )

    undPrice = quotes["underlyingValue"]

    # build the df
    df = pd.DataFrame(flat_data)

    df = pd.DataFrame(
        [
            {
                "nse_symbol": symbol,
                "ib_symbol": symbol,
                "instrument": quotes.get("stocks")[i]
                .get("metadata")
                .get("instrumentType"),
                "expiry": quotes.get("stocks")[i].get("metadata").get("expiryDate"),
                "undPrice": undPrice,
                "safe_strike": 0,
                "right": quotes.get("stocks")[i].get("metadata").get("optionType")[:1],
                "strike": quotes.get("stocks")[i].get("metadata").get("strikePrice"),
                "dte": np.nan,
                "hv": quotes.get("stocks")[i]
                .get("marketDeptOrderBook")
                .get("otherInfo")
                .get("annualisedVolatility"),
                "iv": quotes.get("stocks")[i]
                .get("marketDeptOrderBook")
                .get("otherInfo")
                .get("impliedVolatility"),
                "lot": lot,
                "price": quotes.get("stocks")[i].get("metadata").get("lastPrice"),
            }
            for i in range(len(quotes))
        ]
    )

    # Convert nse_symbol to symbol
    df = df.assign(ib_symbol=nse2ib(df.nse_symbol))

    # Convert expiry to UTC NSE eod
    df = df.assign(
        expiry=df.expiry.apply(lambda x: convert_to_utc_datetime(x, eod=True))
    )

    df = df.assign(dte=get_dte(df.expiry))

    # Convert the rest to numeric
    df = df.apply(convert_to_numeric)

    # Convert to %ge
    df.iv = df.iv / 100
    df.hv = df.hv / 100

    # Change instrument type
    instrument_dict = {
        "Stock": "STK",
        "Options": "OPT",
        "Currency": "FX",
        "Index": "IDX",
        "Futures": "FUT",
    }

    inst = df.instrument.str.split()

    s = inst.apply(lambda x: "".join(instrument_dict[item] for item in x))

    df = df.assign(instrument=s)

    df = df[df.instrument.isin(["IDXOPT", "STKOPT"])]

    return df



### Set constants

In [7]:
# --- SET CONSTANTS  ---
# ----------------------
config = load_config()

MARKET = "NSE"

PUTSTDMULT = config.get("PUTSTDMULT")
CALLSTDMULT = config.get("CALLSTDMULT")
MINEXPROM = config.get("MINEXPROM")
MARGINPERORDER = config.get("MARGINPERORDER")

# maps  for nifty and bank nifty
IDXHISTSYMMAP = config.get("IDXHISTSYMMAP")

# IBKR port access
PORT = port = config.get("PORT")
# PAPER = config.get('PAPER_PORT')

# Classes

## Tools (Utilities) Class

In [8]:
class Timer:
    """Timer providing elapsed time"""

    def __init__(self, name: str = "") -> None:
        self.name = name
        self._start_time = None

    def start(self):
        """Start a new timer"""
        if self._start_time is not None:
            raise Exception(f"Timer is running. Use .stop() to stop it")

        now = datetime.now()

        print(f'\n{self.name} started at {now.strftime("%d-%b-%Y %H: %M:%S")}')

        self._start_time = datetime.now()

    def stop(self) -> None:
        if self._start_time is None:
            raise Exception(f"Timer is not running. Use .start() to start it")

        elapsed_time = datetime.now() - self._start_time

        # Extract hours, minutes, seconds from the timedelta object
        hours = elapsed_time.seconds // 3600
        minutes = (elapsed_time.seconds % 3600) // 60
        seconds = elapsed_time.seconds % 60

        print(f"\n...{self.name} took: " + f"{hours:02d}:{minutes:02d}:{seconds:02d}\n")

        self._start_time = None

## NSE specific Classes

In [9]:
class NSEfnos:
    """Class for All NSE FNOS, including Indexes"""

    time_out = 5
    base_url = "https://www.nseindia.com/api"
    page_url = "https://www.nseindia.com/get-quotes/equity?symbol=LT"
    _routes = {
        "stock_meta": "/equity-meta-info",
        "stock_quote": "/quote-equity",
        "stock_derivative_quote": "/quote-derivative",
        "market_status": "/marketStatus",
        "chart_data": "/chart-databyindex",
        "market_turnover": "/market-turnover",
        "equity_derivative_turnover": "/equity-stock",
        "all_indices": "/allIndices",
        "live_index": "/equity-stockIndices",
        "index_option_chain": "/option-chain-indices",
        "equity_option_chain": "/option-chain-equities",
        "currency_option_chain": "/option-chain-currency",
        "pre_open_market": "/market-data-pre-open",
        "holiday_list": "/holiday-master?type=trading",
        "stock_history": "/historical/cm/equity",  # added by rkv
    }

    def __init__(self):
        self.s = requests.Session()
        h = {
            "Host": "www.nseindia.com",
            "Referer": "https://www.nseindia.com/get-quotes/equity?symbol=SBIN",
            "X-Requested-With": "XMLHttpRequest",
            "pragma": "no-cache",
            "sec-fetch-dest": "empty",
            "sec-fetch-mode": "cors",
            "sec-fetch-site": "same-origin",
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.132 Safari/537.36",
            "Accept": "*/*",
            "Accept-Encoding": "gzip, deflate, br",
            "Accept-Language": "en-GB,en-US;q=0.9,en;q=0.8",
            "Cache-Control": "no-cache",
            "Connection": "keep-alive",
        }
        self.s.headers.update(h)
        self.s.get(self.page_url)

    def get(self, route, payload={}):
        url = self.base_url + self._routes[route]
        r = self.s.get(url, params=payload)
        return r.json()

    @live_cache
    def stock_quote(self, symbol):
        data = {"symbol": symbol}
        return self.get("stock_quote", data)

    @live_cache
    def stock_quote_fno(self, symbol):
        data = {"symbol": symbol}
        return self.get("stock_derivative_quote", data)

    @live_cache
    def trade_info(self, symbol):
        data = {"symbol": symbol, "section": "trade_info"}
        return self.get("stock_quote", data)

    @live_cache
    def market_status(self):
        return self.get("market_status", {})

    @live_cache
    def chart_data(self, symbol, indices=False):
        data = {"index": symbol + "EQN"}
        if indices:
            data["index"] = symbol
            data["indices"] = "true"
        return self.get("chart_data", data)

    @live_cache
    def tick_data(self, symbol, indices=False):
        return self.chart_data(symbol, indices)

    @live_cache
    def market_turnover(self):
        return self.get("market_turnover")

    @live_cache
    def eq_derivative_turnover(self, type="allcontracts"):
        data = {"index": type}
        return self.get("equity_derivative_turnover", data)

    @live_cache
    def all_indices(self):
        return self.get("all_indices")

    def live_index(self, symbol="NIFTY 50"):
        data = {"index": symbol}
        return self.get("live_index", data)

    @live_cache
    def index_option_chain(self, symbol="NIFTY"):
        data = {"symbol": symbol}
        return self.get("index_option_chain", data)

    @live_cache
    def equities_option_chain(self, symbol):
        data = {"symbol": symbol}
        return self.get("equity_option_chain", data)

    @live_cache
    def currency_option_chain(self, symbol="USDINR"):
        data = {"symbol": symbol}
        return self.get("currency_option_chain", data)

    @live_cache
    def live_fno(self):
        return self.live_index("SECURITIES IN F&O")

    @live_cache
    def pre_open_market(self, key="NIFTY"):
        data = {"key": key}
        return self.get("pre_open_market", data)

    @live_cache
    def holiday_list(self):
        return self.get("holiday_list", {})

    @live_cache
    def stock_history(self, symbol, days: int = 365, chunks: int = 50):

        date_ranges = make_date_range_for_stock_history(symbol, days, chunks)

        result = []
        for dr in date_ranges:
            result.append(self.get("stock_history", dr))

        df = clean_stock_history(result)

        return df

    def equities(self, sort_me: bool=True) -> set:
        
        equities_data = self.live_fno()
        equities = [kv.get("symbol") for kv in equities_data.get("data")]
        if sort_me:
            equities.sort()

        return set(equities)

    def indexes(self):
        x = "NIFTY,BANKNIFTY,MIDCPNIFTY,NIFTYNXT50,FINNIFTY"
        return set(x.split(","))


class IDXHistories:

    time_out = 5
    base_url = "https://niftyindices.com"
    idx_symbols = IDXHISTSYMMAP.values()
    url = "https://niftyindices.com/Backpage.aspx/getHistoricaldatatabletoString"

    # prepare `post` header
    post_header = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/91.0.4472.77 Safari/537.36",
        "Connection": "keep-alive",
        "sec-ch-ua": '" Not;A Brand";v="99", "Google Chrome";v="91", "Chromium";v="91"',
        "Accept": "application/json, text/javascript, */*; q=0.01",
        "DNT": "1",
        "X-Requested-With": "XMLHttpRequest",
        "sec-ch-ua-mobile": "?0",
        "Content-Type": "application/json; charset=UTF-8",
        "Origin": "https://niftyindices.com",
        "Sec-Fetch-Site": "same-origin",
        "Sec-Fetch-Mode": "cors",
        "Sec-Fetch-Dest": "empty",
        "Referer": "https://niftyindices.com/reports/historical-data",
        "Accept-Language": "en-US,en;q=0.9,hi;q=0.8",
    }

    def __init__(self, days: int = 365) -> None:
        self.s = requests.Session()

        # update session with default headers and get the cookies
        init_header = requests.utils.default_headers()
        init_header.update(
            {
                "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                "AppleWebKit/537.36 (KHTML, like Gecko) "
                "Chrome/91.0.4472.77 Safari/537.36",
            }
        )
        self.s.headers.update(init_header)
        c = self.s.get(url=self.url)
        self.cookies = c.cookies

    def get(self, payload={}):

        r = self.s.post(
            url=self.url,
            headers=self.post_header,
            cookies=self.cookies,
            data=payload,
            timeout=self.time_out,
        )

        return r.json()

    def make_histories(self, days: int = 365, chunks: int = 50):
        """Makes histories for NIFTY50 and BANKNIFTY, based on number of days provided"""

        date_ranges = split_dates(days=days, chunks=chunks)

        # idx_symbols = ["Nifty Bank", "Nifty 50"]

        # organize the payloads
        payloads = [
            {
                "cinfo": str(
                    {
                        "name": idx_symbol,
                        "startDate": s.strftime("%d-%b-%Y"),
                        "endDate": e.strftime("%d-%b-%Y"),
                        "indexName": idx_symbol,
                    }
                )
            }
            for s, e in date_ranges
            for idx_symbol in self.idx_symbols
        ]
        # get the raw jsons
        results = []

        for payload in tqdm(payloads):
            r = self.get(payload=json.dumps(payload))
            results.append(r)

        df = clean_index_history(results)

        return df


def rbi_tr_to_json(wrapper):
    trs = wrapper.find_all("tr")
    op = {}
    for tr in trs:
        tds = tr.find_all("td")
        if len(tds) >= 2:
            key = tds[0].text.strip()
            val = tds[1].text.replace(":", "").replace("*", "").replace("#", "").strip()

            op[key] = val
    return op


class RBI:
    base_url = "https://www.rbi.org.in/"

    def __init__(self):
        self.s = requests.Session()

    def current_rates(self):
        r = self.s.get(self.base_url)

        bs = BeautifulSoup(r.text, "html.parser")
        wrapper = bs.find("div", {"id": "wrapper"})

        return rbi_tr_to_json(wrapper)

    def repo_rate(self):

        rate = self.current_rates().get("Policy Repo Rate")[:-1]

        return float(rate)

## ibfunc Classes

In [10]:
def empty_the_df(df):
    """Empty the dataclass df"""
    empty_df = pd.DataFrame([df.__dict__]).iloc[0:0]
    return empty_df


@dataclass
class OpenOrder:
    """
    Open order template with Dummy data. Use:\n
    `df = OpenOrder().empty()`
    """

    conId: int = 0
    symbol: str = "Dummy"
    secType: str = "STK"
    expiry: datetime = datetime.now()
    strike: float = 0.0
    right: str = "?"  # Will be 'P' for Put, 'C' for Call
    orderId: int = 0
    order: Order = None
    permId: int = 0
    action: str = "SELL"  # 'BUY' | 'SELL'
    totalQuantity: float = 0.0
    lmtPrice: float = 0.0
    status: str = None

    def empty(self):
        return empty_the_df(self)


@dataclass
class Portfolio:
    """
    Portfolio template with Dummy data. Use:\n
    `df = OpenOrder().empty()`
    """

    conId: int = 0
    symbol: str = "Dummy"
    secType: str = "STK"
    expiry: datetime = datetime.now()
    strike: float = 0.0
    right: str = "?"  # Will be 'P' for Put, 'C' for Call
    position: float = 0.0
    mktPrice: float = 0.0
    mktVal: float = 0.0
    avgCost: float = 0.0
    unPnL: float = 0.0
    rePnL: float = 0.0

    def empty(self):
        return empty_the_df(self)

# BIG TEST (`run.py`)

## Consolidate nakeds pickles and put them to history

In [11]:
# Check if there is anything in the `raw` data folder [utils.py]

def handle_raws(): 
    raw_files = get_files_from_patterns()
    if raw_files:
        ans = yes_or_no("Do you want to archive raw nakeds?")
    
        if ans:
            remove_raw_nakeds(save=True)

    else:
        print("No raw files to archive")


handle_raws()

No raw files to archive


In [12]:
# get set of executed symbols from df_naked
symbols_list = [get_pickle(ROOT / "data" / "df_nakeds.pkl").nse_symbol.to_list()]
executed = set(chain.from_iterable(symbols_list)) # takes care of nested lists

# get open order symbols
with IB().connect(port=port, clientId=10) as ib:

    trades = ib.reqAllOpenOrders()
    df_pf = quick_pf(ib)
    try:
        df_openords = get_open_orders(ib)
        open_orders = set(df_openords.symbol.to_list())
    except UnboundLocalError as e:
        logger.info(f"No open orders found")
        open_orders = set()  # empty

# Build the fnos list
# ... initialize fno class
nse = NSEfnos()

fnos = ((set(["BANKNIFTY", "NIFTY"]) | nse.equities()) - executed) - open_orders
fnos = list(fnos - set(nse_ban_list()))
fnos.sort()

## Make Earliest Nakeds
<span style='color:red'><b>Do `Esc+r+y` to convert following to code and build fresh nakeds<br>
Do `Esc+r` to make it back to raw mode</b></span>

In [ ]:
# Build the earliest naked options

if fnos:
    df_nakeds = make_earliest_nakeds(fnos, save=True)

    out = {
        "current_#fnos": len(fnos),
        "unique_symbols": len(df_nakeds.ib_symbol.unique()),
        "# of orders": len(df_nakeds),
    }
else:
    df = get_pickle(ROOT/"data"/"df_nakeds.pkl")
    df_nakeds = arrange_orders(df, maxmargin=MARGINPERORDER)

## Placing the orders

In [ ]:
# --- PREPARE ORDERS

def make_ib_orders(df: pd.DataFrame) -> tuple:
    """Make (contract, order) tuples"""

    contracts = df.contract.to_list()
    orders = [
        LimitOrder(action="SELL", totalQuantity=abs(int(q)), lmtPrice=p)
        for q, p in zip(df.lot, df.xPrice)
    ]

    cos = tuple((c, o) for c, o in zip(contracts, orders))

    return cos

In [ ]:
df = get_pickle(ROOT/"data"/"df_nakeds.pkl")
df_nakeds = arrange_orders(df, maxmargin=MARGINPERORDER)

# cos = make_ib_orders(df_nakeds)
make_ib_orders(df_nakeds)[:2]

## PLACE THE ORDER

## Checking the orders placed

In [17]:
with IB().connect(port=port, clientId=10) as ib:
    df_openords = get_open_orders(ib)

In [20]:
m = df_openords.symbol == 'PEL'
df_openords[m]

,conId,symbol,secType,expiry,strike,right,orderId,order,permId,action,totalQuantity,lmtPrice,status
0,712930003,PEL,OPT,2024-08-29,1050.0,C,84644,"Order(orderId=84644, clientId=1, permId=696504...",696504735,SELL,750.0,137.55,PreSubmitted
246,707116646,PEL,OPT,2024-08-29,900.0,P,84935,"Order(orderId=84935, clientId=1, permId=696505...",696505026,SELL,750.0,68.55,PreSubmitted
276,709219458,PEL,OPT,2024-08-29,1100.0,C,84961,"Order(orderId=84961, clientId=1, permId=696505...",696505052,SELL,750.0,81.00,PreSubmitted


# Modifying the order price

In [21]:
from ib_async import Contract

In [24]:
df_openords.iloc[0].order, int(df_openords.iloc[0].conId)

(Order(orderId=84644, clientId=1, permId=696504735, action='SELL', totalQuantity=750.0, orderType='LMT', lmtPrice=137.55, auxPrice=0.0, tif='DAY', ocaType=3, displaySize=2147483647, rule80A='0', openClose='', volatilityType=0, deltaNeutralOrderType='None', referencePriceType=0, account='U9329809', clearingIntent='IB', adjustedOrderType='None', cashQty=0.0, dontUseAutoPriceForHedge=True),
 712930003)

In [30]:
order, conId = df_openords.iloc[0].order, int(df_openords.iloc[0].conId)
order.lmtPrice = order.lmtPrice + 0.75

with IB().connect(port=port, clientId=10) as ib:
    contract = ib.qualifyContracts(Contract(conId=conId))[0]
    p = ib.placeOrder(contract, order)

In [37]:
p.order.lmtPrice - 137.55

2.25

In [38]:
(139.8-137.55)/0.75

3.0

# Miscellaneous

## Function Test Bed

In [ ]:
# --- TEST BED ---
# ----------------


def cancel_open_orders(ib) -> pd.DataFrame:
    """!!!Not working!!! --- CHECK"""

    trades = ib.reqAllOpenOrders()  # To kickstart collection of open orders
    ib.sleep(0.3)
    trades = ib.trades()  # Get the trades

    orders = {
        t.order
        for t in trades
        if t.orderStatus.status == "Submitted"
        if t.order.action == "SELL"
    }

    # df_open_orders = get_open_orders(ib)
    # ords = df_open_orders.order.to_list()

    BLK = 25
    ords = list(orders)
    o_blk = [ords[i : i + BLK] for i in range(0, len(ords), BLK)]

    cancels = []

    for ob in o_blk:
        cancels.append([ib.cancelOrder(o) for o in ob])
        ib.sleep(0.3)

    return cancels

## Stock Price (Stock Only! NOT FOR INDEX)

In [ ]:
symbol = "RELIANCE"
q = nse.stock_quote(symbol)

stock_price = q.get("priceInfo").get("lastPrice")

stock_price